# Media

In [31]:
import datetime
print(f"Last updated: {str(datetime.datetime.now())[:10]}")

Last updated: 2024-08-30


Since around 2020, I have been keeping track of almost all media I consume - mostly books and games, but also concerts (but not CD's), movies, museums, and other forms of culture. Here, you can find a quick summary of what I've been up to

## Last enjoyed

In [32]:
import pandas as pd
df = pd.read_csv("media.csv")

verb_dict = {"Book":"read","Game":"played","Movie":"watched","Concert":"visited","Museum":"visited"}
end = f" by {df['creator'].iloc[-1]}" if not any(df['creator'].iloc[-1:].isna()) else ""
print(f"Last {verb_dict.get(df['type'].iloc[-1], 'visited')}: {df['name'].iloc[-1]}{end}")
score_dict = {10:'close to perfect!', 9:'fantastic!', 8:'great', 7:'good', 6:'okay', 5:'meh...', 4:'not great', 3:'bad', 2:'terrible', 1: 'abhorrent', 0:'the worst!'}
print(f"Which I thought was: {score_dict[round(df['score'].iloc[-1])]}")

Last played: Stray Gods: The Roleplaying Musical by Summerfall Studios
Which I thought was: great


## All-time favorites

In [33]:
df3 = df.copy()
sample = df3[df3['score'] >= 9.5].sample(5).reset_index()
print("Some works I can wholeheartedly recommend are:")
for i in range(len(sample)):
    print(f"* {sample['name'][i]} by {sample['creator'][i]}, which I rated at {sample['score'][i]}")

Some works I can wholeheartedly recommend are:
* Stories of Your Life and Others by Ted Chiang, which I rated at 10.0
* Roshambo, of Waarom we Steen-Papier-Schaar Spelen by DraeCKI, which I rated at 9.9
* The MANIAC by Benjamin Labatut, which I rated at 9.8
* Footprints Festival 2023 by TivoliVredenburg, which I rated at 9.5
* Deltarune: Chapter 2 by Toby Fox, which I rated at 10.0


## What?

In [34]:
import plotly.express as px
def rounder(score):
    score = float(round(score*2))/2
    #if score == 10:
    #    return "score≈" + str(9.9)
    #return "score≈" + str(score)
    return score

def counter(type, score):
    return len(df4[(df4['type'] == type) & (df4['score'] == score)])

df4 = df.copy()
df4[['year','month']] = df4['date'].str.split('-',expand=True)
#df4 = df4.groupby(['type', 'score']).size().rename_axis(['type', 'score']).reset_index(name='counts')
df4['counts'] = df4.apply(lambda x: counter(x.type, x.score), axis=1)
df4['scoreactual'] = df4['score'].apply(lambda x: rounder(x))
df4 = df4.sort_values(by=['type', 'score'])
fig = px.sunburst(df4, path=['type', 'year', 'scoreactual', 'name'], 
                  labels=[''], maxdepth=2,
                    hover_data={
                    })
fig.update_layout(margin={'t':5,'l':5,'b':5,'r':5}, showlegend=False)
fig.update_traces(sort=False, hovertemplate='<b>Amount: %{value} </b><br>%{label}')
fig.show()

## How much?

In [35]:
import plotly.express as px
import numpy as np

df2 = df.copy()
df2 = df2.groupby(['type', 'date']).size().rename_axis(['type', 'date']).reset_index(name='counts')
df2[['year','month']] = df2['date'].str.split('-',expand=True)
df2 = df2.drop(columns=['date'])
types = []; counts = []; years = []; months = []
for year in range(min(df2['year'].astype(int)), max(df2['year'].astype(int)) + 1):
    for month in range(1, 13):
        actualmonth = '0' * (2 - len(str(month))) + str(month)
        for type in df2['type'].unique():
            if not ((df2['year'] == str(year)) & (df2['month'] == actualmonth) & (df2['type'] == type)).any():
                types.append(type); counts.append(0); years.append(str(year)); months.append(actualmonth)
for year in range(min(df2['year'].astype(int)), max(df2['year'].astype(int)) + 1):
    for month in range(1, 13):
        actualmonth = '0' * (2 - len(str(month))) + str(month)
        types.append("All"); counts.append(df2[(df2['year'] == str(year)) & (df2['month'] == actualmonth)]['counts'].sum()); years.append(str(year)); months.append(actualmonth)
added_rows = pd.DataFrame({'type':types, 'counts':counts, 'year':years, 'month':months})
df2 = pd.concat([df2, added_rows], ignore_index=True)
types = []; counts = []; years = []; months = []
for month in range(1, 13):
    actualmonth = '0' * (2 - len(str(month))) + str(month)
    for type in df2['type'].unique():
        count = sum(df2[(df2['type'] == type) & (df2['month'] == actualmonth)]['counts']) / len(df2['year'].unique())
        count = count if not np.isnan(count) else 0
        types.append(type); counts.append(count); years.append("Mean"); months.append(actualmonth)
added_rows = pd.DataFrame({'type':types, 'counts':counts, 'year':years, 'month':months})
df2 = pd.concat([df2, added_rows], ignore_index=True)
df2 = df2.sort_values(by=['year', 'month'])
fig = px.line(df2, x='month', y='counts', color='year', animation_frame='type',
              color_discrete_map={"Mean": 'black'}, custom_data=['year'], template='plotly_white',
              labels={
                     "month": "Month",
                     "counts": "Amount",
                     "year": "Year"
                 },
                 hover_data={
                            'type':False,
                            'month':False,
                            'counts':False
                        })
fig.update_layout(margin={'t':10,'l':0,'b':0,'r':0}, yaxis_range=[0,15], xaxis_range=[0,11],
                  xaxis = dict(
                    tickmode = 'array',
                    tickvals = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'],
                    ticktext = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
                ), showlegend=False)
fig.update_traces(opacity=.4)
fig.update_traces(selector=dict(line_color="black"), opacity=1)
fig.update_traces(hovertemplate='%{customdata[0]}<extra></extra>')
fig['layout']['sliders'][0]['pad']=dict(r= 10, t= 20, b=20)
fig["layout"].pop("updatemenus")
fig.show()


## Albums
One particular interest I have is collecting albums out of Rolling Stone's Top 500 Albums list. Below, you can see how many I already have (out of the 2020 list).

In [36]:
albums_df = pd.read_csv("albums.csv", sep=';')
albums_df['Rounded year'] = albums_df['Year'].apply(lambda x: 10 * round(int(x) / 10))
albums_df['Have'] = albums_df['Have'].apply(lambda x: 1 if x == "X" else x)
score = len(albums_df[(albums_df['Have'] == 1)]) / len(albums_df)
displays = int(score * 40)
print(f"Total:     [{'+' * displays}{'.' * (40 - displays)}] {len(albums_df[(albums_df['Have'] == 1)])} out of {len(albums_df)}\n")
for year in sorted(albums_df['Rounded year'].unique()):
    score = len(albums_df[(albums_df['Rounded year'] == year) & (albums_df['Have'] == 1)]) / len(albums_df[albums_df['Rounded year'] == year])
    symbol = "⋆" if score == 1 else "+"
    displays = int(score * 40)
    print(f"{year-5}-{year+4}: [{symbol * displays}{'.' * (40 - displays)}] {len(albums_df[(albums_df['Rounded year'] == year) & (albums_df['Have'] == 1)])} out of {len(albums_df[albums_df['Rounded year'] == year])}")

Total:     [+++++++++...............................] 119 out of 500

1955-1964: [++......................................] 2 out of 28
1965-1974: [+++++++++...............................] 36 out of 146
1975-1984: [+++++++++...............................] 25 out of 106
1985-1994: [+++++++++++.............................] 26 out of 88
1995-2004: [+++++++++...............................] 20 out of 83
2005-2014: [++++++++++..............................] 8 out of 31
2015-2024: [++++....................................] 2 out of 18


In [37]:
albums_df['Have'] = albums_df['Have'].cumsum()
albums_df = albums_df[albums_df['Have'].notnull()]
fig = px.line(albums_df, x='#', y='Have', template='plotly_white',
              labels={
                     "#": "Number of albums in top 500",
                     "Have": "Amount in collection",
                 },)
fig.update_layout(margin={'t':10,'l':0,'b':0,'r':10}, showlegend=False)
fig.show()